In [1]:
import numpy as np
import pandas as pd
import random
import json
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sixni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sixni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
with open('intents2.json') as file:
    intents = json.load(file)

lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))


In [3]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])


In [4]:
from tensorflow.keras.optimizers import legacy

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Using the legacy SGD optimizer
sgd = legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)



Epoch 1/200


151/151 [==============================] - 1s 855us/step - loss: 4.0160 - accuracy: 0.0360
Epoch 2/200
151/151 [==============================] - 0s 790us/step - loss: 3.9180 - accuracy: 0.0493
Epoch 3/200
151/151 [==============================] - 0s 812us/step - loss: 3.8405 - accuracy: 0.0639
Epoch 4/200
151/151 [==============================] - 0s 782us/step - loss: 3.7636 - accuracy: 0.0746
Epoch 5/200
151/151 [==============================] - 0s 820us/step - loss: 3.6156 - accuracy: 0.0905
Epoch 6/200
151/151 [==============================] - 0s 782us/step - loss: 3.5400 - accuracy: 0.1318
Epoch 7/200
151/151 [==============================] - 0s 801us/step - loss: 3.3395 - accuracy: 0.1571
Epoch 8/200
151/151 [==============================] - 0s 806us/step - loss: 3.2132 - accuracy: 0.1824
Epoch 9/200
151/151 [==============================] - 0s 800us/step - loss: 3.0666 - accuracy: 0.2210
Epoch 10/200
151/151 [==============================] - 0s 823us/step 

151/151 [==============================] - 0s 815us/step - loss: 0.7559 - accuracy: 0.7750
Epoch 74/200
151/151 [==============================] - 0s 814us/step - loss: 0.8109 - accuracy: 0.7723
Epoch 75/200
151/151 [==============================] - 0s 789us/step - loss: 0.7179 - accuracy: 0.7723
Epoch 76/200
151/151 [==============================] - 0s 803us/step - loss: 0.8612 - accuracy: 0.7364
Epoch 77/200
151/151 [==============================] - 0s 809us/step - loss: 0.8107 - accuracy: 0.7563
Epoch 78/200
151/151 [==============================] - 0s 814us/step - loss: 0.7216 - accuracy: 0.7843
Epoch 79/200
151/151 [==============================] - 0s 806us/step - loss: 0.7603 - accuracy: 0.7763
Epoch 80/200
151/151 [==============================] - 0s 814us/step - loss: 0.8484 - accuracy: 0.7457
Epoch 81/200
151/151 [==============================] - 0s 822us/step - loss: 0.8185 - accuracy: 0.7390
Epoch 82/200
151/151 [==============================] - 0s 818us/step - loss:

151/151 [==============================] - 0s 849us/step - loss: 0.6598 - accuracy: 0.8189
Epoch 152/200
151/151 [==============================] - 0s 813us/step - loss: 0.6807 - accuracy: 0.8123
Epoch 153/200
151/151 [==============================] - 0s 838us/step - loss: 0.6596 - accuracy: 0.7923
Epoch 154/200
151/151 [==============================] - 0s 814us/step - loss: 0.5601 - accuracy: 0.8336
Epoch 155/200
151/151 [==============================] - 0s 824us/step - loss: 0.5428 - accuracy: 0.8349
Epoch 156/200
151/151 [==============================] - 0s 826us/step - loss: 0.6788 - accuracy: 0.8136
Epoch 157/200
151/151 [==============================] - 0s 833us/step - loss: 0.5393 - accuracy: 0.8269
Epoch 158/200
151/151 [==============================] - 0s 834us/step - loss: 0.6806 - accuracy: 0.8069
Epoch 159/200
151/151 [==============================] - 0s 811us/step - loss: 0.5566 - accuracy: 0.8349
Epoch 160/200
151/151 [==============================] - 0s 830us/ste

In [5]:
# Further training the model for an additional 100 epochs
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)


Epoch 1/100
151/151 [==============================] - 0s 814us/step - loss: 0.7261 - accuracy: 0.8109
Epoch 2/100
151/151 [==============================] - 0s 809us/step - loss: 0.7057 - accuracy: 0.7936
Epoch 3/100
151/151 [==============================] - 0s 789us/step - loss: 0.7176 - accuracy: 0.8043
Epoch 4/100
151/151 [==============================] - 0s 801us/step - loss: 0.6639 - accuracy: 0.8176
Epoch 5/100
151/151 [==============================] - 0s 808us/step - loss: 0.6924 - accuracy: 0.8043
Epoch 6/100
151/151 [==============================] - 0s 801us/step - loss: 0.8137 - accuracy: 0.7923
Epoch 7/100
151/151 [==============================] - 0s 800us/step - loss: 0.6509 - accuracy: 0.8043
Epoch 8/100
151/151 [==============================] - 0s 786us/step - loss: 0.7192 - accuracy: 0.7989
Epoch 9/100
151/151 [==============================] - 0s 809us/step - loss: 0.7025 - accuracy: 0.8003
Epoch 10/100
151/151 [==============================] - 0s 794us/step - l

151/151 [==============================] - 0s 816us/step - loss: 0.6768 - accuracy: 0.7856
Epoch 80/100
151/151 [==============================] - 0s 821us/step - loss: 0.6193 - accuracy: 0.8242
Epoch 81/100
151/151 [==============================] - 0s 836us/step - loss: 0.8348 - accuracy: 0.7963
Epoch 82/100
151/151 [==============================] - 0s 817us/step - loss: 0.6648 - accuracy: 0.8123
Epoch 83/100
151/151 [==============================] - 0s 819us/step - loss: 0.7531 - accuracy: 0.8083
Epoch 84/100
151/151 [==============================] - 0s 804us/step - loss: 0.7581 - accuracy: 0.7963
Epoch 85/100
151/151 [==============================] - 0s 808us/step - loss: 0.6114 - accuracy: 0.8162
Epoch 86/100
151/151 [==============================] - 0s 810us/step - loss: 0.7916 - accuracy: 0.7896
Epoch 87/100
151/151 [==============================] - 0s 827us/step - loss: 0.7851 - accuracy: 0.8029
Epoch 88/100
151/151 [==============================] - 0s 800us/step - loss:

In [6]:
model.save('chatbot_model.h5', overwrite=True)


C:\Users\sixni\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s: 
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print("Your bot is ready to talk! Type 'quit' to exit.")
while True:
    message = input("")
    if message.lower() == "quit":
        break
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(res)


Your bot is ready to talk! Type 'quit' to exit.
Hi
1/1 [==============================] - 0s 60ms/step
Hi there. What brings you here today?
quit


In [8]:
# Save the model before starting the chat interaction
model.save('chatbot_model.h5', overwrite=True)

# Fabricated Test Sentences and Expected Labels
test_sentences = [
# Greeting
("Hi, how's it going?", "greeting"),
("Hello, anyone there?", "greeting"),
("Hey there!", "greeting"),
("Howdy, partner!", "greeting"),
("Bonjour, how are you?", "greeting"),

# Morning
("Hope you're having a good morning.", "morning"),
("What a beautiful morning!", "morning"),
("Morning has broken.", "morning"),
("Good morning to everyone!", "morning"),
("Woke up feeling great this morning!", "morning"),

# Afternoon
("Having a lazy afternoon here.", "afternoon"),
("Good afternoon, how can I help?", "afternoon"),
("It's a sunny afternoon.", "afternoon"),
("Hope your afternoon is going well.", "afternoon"),
("Afternoon delight!", "afternoon"),

# Evening
("What a peaceful evening.", "evening"),
("Good evening, friends.", "evening"),
("The evening is young.", "evening"),
("Hope you're enjoying your evening.", "evening"),
("Evening's greetings.", "evening"),

# Night
("Wishing you a good night.", "night"),
("Nighty night!", "night"),
("See you in the morning, goodnight!", "night"),
("Don't let the bedbugs bite, good night!", "night"),
("Have a peaceful night.", "night"),

# Goodbye
("Catch you on the flip side.", "goodbye"),
("I'm off, bye!", "goodbye"),
("Goodbye and good luck!", "goodbye"),
("It's been real, bye!", "goodbye"),
("See you later, alligator.", "goodbye"),

# Thanks
("Really appreciate it, thanks!", "thanks"),
("Thank you for your support.", "thanks"),
("Couldn't have done it without you, thanks!", "thanks"),
("Thanks a ton!", "thanks"),
("Your help means a lot, thank you!", "thanks"),

# No-response
("...", "no-response"),
("     ", "no-response"),
("", "no-response"),
("....", "no-response"),
("    ", "no-response"),

# Neutral-response
("Just the usual, nothing much.", "neutral-response"),
("Eh, not much happening.", "neutral-response"),
("Nothing much, you know how it is.", "neutral-response"),
("Same old, same old.", "neutral-response"),
("Not much, just hanging around.", "neutral-response"),
# About
("Who exactly are you?", "about"),
("Can you tell me more about what you are?", "about"),
("I'm curious, who are you?", "about"),
("Could you introduce yourself?", "about"),
("What name do you go by?", "about"),

# Skill
("List your skills for me, please.", "skill"),
("What tasks can you perform?", "skill"),
("I'm wondering about the services you provide.", "skill"),
("In what ways can you be of assistance?", "skill"),


# Name
("You can call me John.", "name"),
("My name's Jane, pleased to meet you.", "name"),
("I go by Alex.", "name"),
("Just call me Sam.", "name"),
("I'm Chris.", "name"),

# Help
("I'm in a bit of a bind, can you help out?", "help"),
("I could really use your assistance right now.", "help"),
("Is there any guidance you can offer me?", "help"),
("This problem has me stuck, can you help?", "help"),

# Sad
("Lately, I just feel so alone.", "sad"),
("These days, I'm feeling quite down.", "sad"),
("I've been feeling really sad lately.", "sad"),
("I don't know why, but I feel so lonely.", "sad"),
("It's been tough, I feel quite low.", "sad"),
("The end of the movie left me feeling unexpectedly sad and reflective.", "sad"),
("Saying goodbye is always hard, filling me with a deep sense of sadness.", "sad"),
("The news of his passing brought a profound sadness to the community.", "sad"),
# Stressed
("The stress is constant, and I can't find relief.", "stressed"),
("I'm so stressed, it's affecting my health.", "stressed"),
("Work has been so stressful, I dread going every day.", "stressed"),
("The mounting deadlines have me feeling stressed beyond belief.", "stressed"),
("Feeling stressed, she took a moment to breathe deeply and center herself.", "stressed"),
("Balancing work and personal life is a constant source of stress for me.", "stressed"),
("The approaching deadline has me feeling really stressed.", "stressed"),
("Feeling stressed out with all these responsibilities.", "stressed"),
("I'm so stressed I can barely sleep.", "stressed"),
("Work has been incredibly stressful lately.", "stressed"),
("With the exams coming up, I'm feeling more stressed than ever.", "stressed"),

# Happy
("I'm over the moon with how my project turned out.", "happy"),
("Finding myself on cloud nine after hearing the good news!", "happy"),
("Today just keeps getting better and better!", "happy"),
("Feeling so uplifted after our conversation.", "happy"),
("Absolutely nothing can dampen my spirits today!", "happy"),
("I'm just beaming with joy right now, can't stop smiling.", "happy"),
("Got a spring in my step after the recent success!", "happy"),
("Feeling incredibly blessed with the people around me.", "happy"),
("It seems like everything is finally falling into place.", "happy"),
("I'm filled to the brim with joy today!", "happy"),
("Honestly, this feels like one of the best days ever.", "happy"),
("I can hardly contain my excitement, so much good news!", "happy"),
("Just appreciating how beautiful life is at this moment.", "happy"),
("Thrilled beyond words with how things are going!", "happy"),
("My heart's just overflowing with joy and gratitude.", "happy"),



# Confused
("I've read this over and over but it makes no sense.", "confused"),
("Can someone explain this? I'm lost.", "confused"),
("Nothing adds up, I'm just so confused.", "confused"),
("I'm confused about the instructions, can you explain again?", "confused"),
("Feeling really confused about my feelings right now.", "confused"),
("This puzzle has me completely confused.", "confused"),
("Confused about which path to take for my career.", "confused"),
("Why would they do that? I'm so confused.", "confused"),

# Excited
("This is going to be amazing, I can hardly wait! The anticipation is like a firework show in my heart.", "excited"),
("Just thinking about it fills me with so much excitement. It's like every cell in my body is dancing to the same joyful tune.", "excited"),
("I've been looking forward to this for so long! It feels like waiting for the first drop on a giant roller coaster – thrilling and nerve-wracking!", "excited"),
("The idea of us finally getting together for the event has me over the moon. It's like counting down to Christmas morning as a kid.", "excited"),
("Can you believe it's happening? My excitement is bubbling up like a volcano ready to erupt with joy.", "excited"),
("Every time I think about the upcoming trip, a wave of excitement washes over me. It's like the prelude to an epic adventure.", "excited"),
("The excitement is unreal; I feel like a child on the eve of their birthday, wide-eyed and filled with wonder.", "excited"),
("Anticipating our reunion has me so excited, it's like my heart is playing hopscotch in my chest.", "excited"),
("Imagining stepping onto that stage has me electrified with excitement. It's like my dreams are finally taking flight.", "excited"),
("The countdown to the launch feels like waiting for a supernova explosion. My excitement is cosmic!", "excited"),

# Worried
("I'm worried about the future, it seems so uncertain.", "worried"),
("My mind won't stop worrying about what could go wrong.", "worried"),
("I worry so much about my loved ones, it keeps me up at night.", "worried"),
("I'm constantly worried about the future, wondering if I'm making the right choices.", "worried"),
("His health has me really worried; I wish there was more I could do to help.", "worried"),
("Seeing her so upset and worried about the exams broke my heart.", "worried"),


# Bored
("I've run out of things to do, just bored.", "bored"),
("The monotony is getting to me, feeling so uninterested.", "bored"),
("Everything seems dull, I'm just bored out of my mind.", "bored"),

# Scared
("There's this constant fear, I'm really scared.", "scared"),
("I can't explain it, but I'm just frightened all the time.", "scared"),
("Everything's just so intimidating, I'm scared.", "scared"),
("I'm scared of the unknown, of stepping out of my comfort zone.", "scared"),
("Hearing strange noises at night when I'm alone gets me really scared.", "scared"),
("The thought of losing you is what scares me the most.", "scared"),


# Surprised
("That news caught me off guard, so surprised.", "surprised"),
("I wasn't expecting that at all, totally shocked.", "surprised"),
("This is unbelievable, I'm just so surprised.", "surprised"),
("The party was a total surprise, leaving me speechless and overwhelmed with joy.", "surprised"),
("I was surprised at how much I enjoyed the unexpected gift.", "surprised"),
("His sudden change of plans took everyone by surprise, scrambling to adjust.", "surprised"),
# Curious
("I've always wondered how the universe came into existence; it's a question that fuels my curiosity.", "curious"),
("She had a curious look on her face, eager to learn more about the topic.", "curious"),
("His endless questions about the world around him showed how deeply curious he was.", "curious"),


# Grateful
("I'm incredibly grateful to have friends who support me through thick and thin.", "grateful"),
("After the accident, she felt grateful for every new day she was given.", "grateful"),
("Looking back, I'm grateful for the lessons learned from difficult times.", "grateful"),


# Frustrated
("No matter what I do, it just doesn't work out.", "frustrated"),
("I'm hitting a wall with this problem.", "frustrated"),
("It feels like I'm going in circles, so frustrated.", "frustrated"),
("Feeling frustrated with the constant delays.", "frustrated"),
("I'm so frustrated trying to solve this problem.", "frustrated"),
("It's frustrating when people don't listen.", "frustrated"),
("My efforts seem futile, and it's incredibly frustrating.", "frustrated"),
("The more I try, the more frustrated I become.", "frustrated"),

# Disappointed
("I had high hopes, but things didn't pan out.", "disappointed"),
("It's hard not to feel let down by this outcome.", "disappointed"),
("I thought it would be different, quite disappointed.", "disappointed"),
("I'm really disappointed in how things turned out.", "disappointed"),
("Feeling disappointed that I didn't get the job.", "disappointed"),
("Disappointed with my performance, I know I can do better.", "disappointed"),
("It's so disappointing to see no progress.", "disappointed"),
("After all the hype, the event was a disappointment.", "disappointed"),

# Relieved
("Hearing the good news, I felt a wave of relief wash over me.", "relieved"),
("She was relieved to find out that her family was safe after the storm.", "relieved"),
("Finishing the project on time was a huge relief, lifting the weight off my shoulders.", "relieved"),
("Finally finished my project, feeling so relieved.", "relieved"),
("I was relieved to hear they made it home safely.", "relieved"),
("Feeling relieved after receiving the good news.", "relieved"),
("The moment I found my lost keys, I felt immensely relieved.", "relieved"),
("Relieved that the results came back negative.", "relieved"),


# Hopeful
("I can see the light at the end of the tunnel.", "hopeful"),
("There's a good chance things will improve, feeling hopeful.", "hopeful"),
("Keeping my fingers crossed for a positive outcome.", "hopeful"),


# Calm
("There's a serene quietness in my mind right now.", "calm"),
("I've found my peace, finally feeling calm.", "calm"),
("Everything's so still and peaceful, I'm calm.", "calm"),
("Taking deep breaths helps me stay calm.", "calm"),
("Feeling calm and relaxed after the yoga session.", "calm"),
("The calm before the storm is quite deceiving.", "calm"),
("A calm sea does not make a skilled sailor.", "calm"),
("Keeping calm in stressful situations is a skill.", "calm"),
("I am at peace now.", "calm"),

# Determined
("No matter how hard it gets, I'm determined to reach my goals.", "determined"),
("Her determined attitude was clear; she would not give up without a fight.", "determined"),
("Even in the face of adversity, he remained determined to succeed.", "determined"),
("I will not stop short of reaching my goals.","determined"),
("I will keep going.", "determined"),
("I've set my sights on my goal, and nothing will deter me.", "determined"),
("No obstacle is too big for me; I'm completely determined to overcome it.", "determined"),
("I'm fully committed to my plan and ready to face any challenges.", "determined"),
("My determination to succeed is stronger than any setback I might face.", "determined"),
("Every day, I wake up motivated and determined to make progress towards my goals.", "determined"),
("I have a clear vision of my future, and I'm determined to make it a reality.", "determined"),
("With unwavering focus, I'm determined to achieve my dreams, no matter what.", "determined"),
("My resolve is firm, and my spirit is determined; success is my only option.", "determined"),
("Despite the odds, my determination keeps me going forward.", "determined"),
("I'm determined to turn my dreams into reality, one step at a time.", "determined"),
("This journey requires determination, and I have plenty to spare.", "determined"),
("I'm not just hopeful; I'm determined to make a significant change in my life.", "determined"),
("Fueled by determination, I'm ready to tackle this project head-on.", "determined"),
("My determination is my strength, pushing me closer to my goals.", "determined"),
("I refuse to give up, driven by a deep sense of determination to succeed.", "determined"),


# Jealous
("Seeing them together, I couldn't shake off this jealous feeling eating away at me.", "jealous"),
("I hate to admit it, but I felt jealous when I heard about her promotion.", "jealous"),
("It's hard not to feel jealous when you see others achieving your dreams.", "jealous"),


# Proud
("Seeing my work come to fruition makes me feel incredibly proud.", "proud"),
("I'm so proud of you for overcoming your fears and facing the challenge head-on.", "proud"),
("She beamed with pride as she shared her accomplishments with her family.", "proud"),


# Guilty
("I can't believe I let them down like that.", "guilty"),
("The guilt from my mistake is eating me up.", "guilty"),
("I know I messed up big time, feeling really guilty.", "guilty"),

# Content
("I've finally found a good balance in life, feeling so content.", "content"),
("Nothing beats this feeling of peace and satisfaction.", "content"),
("I'm exactly where I want to be in life, so content.", "content"),
("All is well in my world, completely content.", "content"),
("I'm just enjoying the simple pleasures, feeling very content.", "content"),
("My heart is full, and I'm content with what I have.", "content"),
("I've found joy in the little things, and it makes me content.", "content"),
("I'm satisfied with my progress, feeling content.", "content"),
("Life's challenges seem manageable, and I'm feeling content.", "content"),
("I'm cherishing these moments of calm and contentment.", "content"),
("I don't need more to be happy, I'm content as is.", "content"),
("Feeling content, not because everything is perfect, but because I choose to focus on the positive.", "content"),
("Finding contentment in my daily routine has been a game changer.", "content"),
("There's a warm sense of contentment in being with loved ones.", "content"),
("I'm feeling content, like I'm exactly where I'm supposed to be.", "content"),
("My journey has brought me to a place of contentment.", "content"),
("After a long day, I'm content to just be home and relax.", "content"),
("I've learned to find contentment in the simplicity of life.", "content"),
("This quiet moment alone has filled me with contentment.", "content"),
("I'm content with the decisions I've made - they've led me to a good place.", "content"),


# Regretful
("Looking back, I'm regretful for the chances I didn't take, wishing I had been braver.", "regretful"),
("He was regretful of his harsh words, wishing he could take them back.", "regretful"),
("Feeling regretful, she apologized for missing the important event.", "regretful"),


# Shy
("Just the thought of speaking up makes me anxious.", "shy"),
("I always freeze up around new people, so shy.", "shy"),
("I wish it were easier for me to open up, feeling really shy.", "shy"),
("I've always been shy around new people, taking my time to open up.", "shy"),
("Her shy smile across the room caught my attention, sparking my curiosity.", "shy"),
("Being shy, he preferred writing his thoughts down rather than speaking out loud.", "shy"),

# Confident
("I've got this, no doubt in my mind.", "confident"),
("I'm ready to face whatever comes, feeling confident.", "confident"),
("Believing in myself has made such a difference, so confident.", "confident"),

# Embarrassed
("I'm mortified by what happened, can't stop thinking about it.", "embarrassed"),
("I turned so red, I could feel the embarrassment all over.", "embarrassed"),
("I wish the ground would swallow me whole at that moment.", "embarrassed"),
("So awkward, I just wanted to disappear right then and there.", "embarrassed"),
("I'm dying of embarrassment over what I said.", "embarrassed"),
("Why on earth did I think that was a good idea?", "embarrassed"),
("I can't even look them in the eye now, I'm so embarrassed.", "embarrassed"),
("I'm so ashamed of what I did, I can hardly stand it.", "embarrassed"),
("That was such an embarrassing mistake, I don't know how I'll recover.", "embarrassed"),
("I feel so self-conscious now, like everyone's watching.", "embarrassed"),



# Relaxed
("Spending the day at the beach has me feeling completely relaxed and at peace.", "relaxed"),
("After a long week, a quiet evening at home is all I need to feel relaxed.", "relaxed"),
("The sound of rain against the window is the perfect backdrop for a relaxed afternoon.", "relaxed"),

# Surprised
("Wow, that news really caught me off guard!", "surprised"),
("Never saw that coming, what a shock!", "surprised"),
("This is beyond my wildest expectations!", "surprised"),

# Disgusted
("The way they treated that animal left me feeling completely disgusted.", "disgusted"),
("I read about the corruption in the news and felt a wave of disgust.", "disgusted"),
("Seeing the trash strewn all over the park was utterly disgusting.", "disgusted"),
("I'm disgusted by how people can be so inconsiderate of others.", "disgusted"),
("The smell coming from the abandoned building was so bad, it was disgusting.", "disgusted"),
("Finding out he lied to me about everything made me feel disgusted.", "disgusted"),
("Watching someone be so wasteful with food really disgusts me.", "disgusted"),
("I felt a deep sense of disgust when I saw how they were polluting the river.", "disgusted"),
("The level of greed in that situation is just disgusting to me.", "disgusted"),
("I can’t hide my disgust at the way those people were gossiping maliciously.", "disgusted"),
("Seeing the state of the neglected animals at the shelter filled me with disgust.", "disgusted"),
("I'm disgusted by the lack of respect shown to the elderly in that incident.", "disgusted"),
("The deceit and manipulation in that case are completely disgusting.", "disgusted"),
("I felt disgusted when I learned about the exploitation in the industry.", "disgusted"),
("Watching the video of the polluted ocean made me feel disgusted and sad.", "disgusted"),


# Amused
("This is hilariously clever, I can't stop laughing!", "amused"),
("Never thought I'd find something so amusing!", "amused"),
("I'm totally entertained by this, what a joy!", "amused"),

# Ashamed
("Every time I think about what happened, I feel a deep sense of shame washing over me.", "ashamed"),
("I can't look them in the eyes because I'm so ashamed of what I did.", "ashamed"),
("The guilt is unbearable; I'm completely ashamed of my actions.", "ashamed"),
("I'm so ashamed of what I did yesterday.", "ashamed"),
("Feeling ashamed for not standing up for my friend.", "ashamed"),
("I can't look them in the eye; I'm too ashamed.", "ashamed"),
("Ashamed of failing the test, even after studying so hard.", "ashamed"),
("I was ashamed to admit I was wrong.", "ashamed"),


# Indifferent
("No matter what I do, it all seems pointless, leaving me feeling indifferent about the future.", "indifferent"),
("She shrugged, her expression indifferent, unaffected by the news that thrilled everyone else.", "indifferent"),
("Lately, everything just feels gray and I'm indifferent to things I used to love.", "indifferent"),
("I used to get excited about things easily, but now, nothing seems to matter much.", "indifferent"),
("Everyone else gets happy or upset about stuff, and I'm just here, not feeling much either way.", "indifferent"),
("It's weird, things I loved before just don't do it for me anymore.", "indifferent"),
("When people talk about what makes them happy, I just can't relate. I wish I could feel something.", "indifferent"),
("Every day feels the same. I keep hoping for something to change, but it doesn't.", "indifferent"),
("I don't have strong opinions on things anymore. It's like I don't care as much as I used to.", "indifferent"),
("My friends are moving on with their lives, and I'm happy for them, but inside, I feel left behind.", "indifferent"),
("Music, movies, stuff I used to enjoy just feels like noise now.", "indifferent"),
("Making decisions feels pointless. It's like, does it even matter what I choose?", "indifferent"),



# Impatient
("Can't this go any faster? I'm running out of patience.", "impatient"),
    ("I've been waiting forever, it's starting to irritate me.", "impatient"),
    ("Why does this have to take so long? I need it now!", "impatient"),
    ("This is taking forever!", "impatient"),
    ("How much longer will this take?", "impatient"),
    ("I'm tired of waiting.", "impatient"),
    ("This wait is killing me.", "impatient"),
    ("I'm getting frustrated with how long this is taking.", "impatient"),
    ("I expected this to be much quicker.", "impatient"),
    ("I have no patience left for this.", "impatient"),
    ("Every minute feels like an hour!", "impatient"),
    ("Isn't there a quicker way to do this?", "impatient"),
    ("I'm losing my patience with this delay.", "impatient"),
    ("It's so hard to keep waiting like this.", "impatient"),
    ("I really can't stand waiting any longer.", "impatient"),

# Patient
("I'm trying to be patient with my progress, understanding that growth takes time.", "patient"),
("Being patient in times of delay is challenging, but it teaches you the value of patience.", "patient"),
("He asked me to be patient as he explained the complex concept, ensuring I understood each step.", "patient"),


# Skeptical
("Sounds too good to be true, I'm pretty skeptical.", "skeptical"),
("I have my doubts about this, not convinced yet.", "skeptical"),
("I'll believe it when I see it, remaining skeptical until then.", "skeptical"),
("I'm skeptical about the new plan, not convinced it will address the core issues.", "skeptical"),
("Her skeptical look told me she wasn't buying the explanation.", "skeptical"),
("Despite the hype, he remained skeptical until seeing the results for himself.", "skeptical"),
("I'm skeptical about the new policy actually working.", "skeptical"),
("He gave her a skeptical look, doubting the truth of her story.", "skeptical"),
("Feeling skeptical about the chances of success.", "skeptical"),
("I remain skeptical of their promises until I see some action.", "skeptical"),
("Their explanation was odd, leaving me somewhat skeptical.", "skeptical"),

# Pessimistic
("No matter how I try, I can't shake this pessimistic feeling about the outcome.", "pessimistic"),
("Lately, I've been feeling more pessimistic about the future, struggling to stay hopeful.", "pessimistic"),
("Her pessimistic outlook on the project dampened the team's spirits.", "pessimistic"),

# Lonely
("The silence of the house only amplified my loneliness, making each day harder than the last.", "lonely"),
("Even in a crowd, I can't shake this feeling of being utterly lonely.", "lonely"),
("Loneliness creeps in whenever I stop keeping myself busy, reminding me of the emptiness.", "lonely"),
("Sometimes I feel so lonely, even in a crowded room.", "lonely"),
("Feeling lonely without my friends around.", "lonely"),
("The silence in the house made her feel lonely.", "lonely"),
("Loneliness hits hardest at night.", "lonely"),
("Being far from home can make you feel really lonely.", "lonely"),


# Loved
("Feeling loved and supported by my friends has made all the difference during tough times.", "loved"),
("The outpouring of love on my birthday made me realize how loved and cherished I am.", "loved"),
("Knowing I am loved by you gives me strength I never knew I had.", "loved"),


# Unloved
("It's like no one truly sees me or cares, feeling completely unloved.", "unloved"),
("Feeling neglected and forgotten, as if I don't matter to anyone.", "unloved"),
("The silence from my friends is deafening, feeling very unloved right now.", "unloved"),
("Sometimes, I feel so unloved and overlooked by those around me.", "unloved"),
("Feeling unloved and unappreciated in a relationship is deeply hurtful.", "unloved"),
("The lack of response made her feel unloved, questioning her worth.", "unloved"),

# Hopeless
("Sometimes, the situation feels so hopeless, like there's no light at the end of the tunnel.", "hopeless"),
("I'm struggling to find any hope in this, everything seems so bleak and hopeless.", "hopeless"),
("Feeling hopeless, he wondered if things would ever start to look up again.", "hopeless"),
("Sometimes, the situation feels completely hopeless.", "hopeless"),
("Feeling hopeless about finding a good job.", "hopeless"),
("I'm starting to feel hopeless in my quest for happiness.", "hopeless"),
("When I look at the mess, I feel utterly hopeless.", "hopeless"),
("It's easy to feel hopeless when things keep going wrong.", "hopeless"),




# Desperate
("At my wit's end, feeling completely desperate for a solution.", "desperate"),
("Feeling like I'm out of options, the desperation is overwhelming.", "desperate"),
("So desperate for a change, it feels like I'm stuck in quicksand.", "desperate"),
("Feeling desperate for a solution to this problem.", "desperate"),
("I'm getting desperate; nothing seems to work.", "desperate"),
("Desperate times call for desperate measures.", "desperate"),
("Feeling so desperate, I might just accept any help now.", "desperate"),
("In a desperate attempt to win, he cheated.", "desperate"),
("Feeling desperate for some sign of improvement.", "desperate"),
("In a desperate attempt to win her back, he bought 100 roses.", "desperate"),
("The situation seems hopeless, and I'm feeling quite desperate.", "desperate"),
("Desperate for company, he called up old friends.", "desperate"),
("With time running out, they made a desperate bid for freedom.", "desperate"),




# Energetic
("Full of energy, ready to tackle anything that comes my way!", "energetic"),
("Feeling so active, like I can run a marathon right now!", "energetic"),
("This burst of energy is amazing, ready to take on new challenges!", "energetic"),

# Lazy
("Today, I just feel lazy, wanting to do nothing but lie in bed and watch movies.", "lazy"),
("The rainy weather makes me feel lazy and unmotivated to work on my tasks.", "lazy"),
("Admitting I've been lazy about my responsibilities is the first step to getting back on track.", "lazy"),
("Lacking the energy to get up and tackle the day ahead.", "lazy"),
("I know I have things to do, but I just can't find the will to start.", "lazy"),
("Just the thought of doing any work today is making me want to go back to sleep.", "lazy"),
("I'm just not in the mood to be productive at all today.", "lazy"),
("Feeling zero motivation to get out of bed, even though my to-do list is growing.", "lazy"),
("It's one of those days where even making a cup of coffee feels like a huge task.", "lazy"),
("I'm avoiding my tasks like the plague today, just don't have the drive.", "lazy"),
("Sitting here, knowing I should be productive, but instead, I'm scrolling endlessly on my phone.", "lazy"),
("I have a deadline approaching, yet here I am, doing absolutely nothing about it.", "lazy"),
("The couch has become my best friend today; I can't seem to leave it.", "lazy"),
("Procrastination has taken over, and I'm watching it happen like an outside observer.", "lazy"),
("Every task on my list seems insurmountable right now, so I'm choosing to ignore them all.", "lazy"),
("I keep telling myself I'll start in an hour, and yet, here I am, still not moving.", "lazy"),
("My ambition is at an all-time low today; can't seem to care about any of my projects.", "lazy"),
("I'm fully aware of what I should be doing, but I'm choosing the path of least resistance: doing nothing.", "lazy"),

# Harmful Thoughts
("Feeling really down and starting to have some dangerous thoughts.", "harmful_thoughts"),
("The thought of harming myself has crossed my mind more than once.", "harmful_thoughts"),
("Can't shake this feeling that I might end up hurting someone or myself.", "harmful_thoughts"),
("Lately, the idea of not being around anymore seems comforting.", "harmful_thoughts"),
("I've been feeling trapped and keep thinking of ways out, even if they're harmful.", "harmful_thoughts"),
("Sometimes, I wonder if everyone would be better off without me.", "harmful_thoughts"),
("I've caught myself thinking about self-harm as a relief from pain.", "harmful_thoughts"),
("The thought of driving off the road entered my mind more than it should.", "harmful_thoughts"),
("Finding it hard to stop thinking about doing something extreme to myself.", "harmful_thoughts"),
("I keep fantasizing about ways to end my pain permanently.", "harmful_thoughts"),
("It's getting more difficult to ignore the thoughts of self-harm.", "harmful_thoughts"),
("I'm scared because I'm starting to see self-harm as a solution.", "harmful_thoughts"),
("The urge to hurt myself is becoming overwhelming.", "harmful_thoughts"),
("I'm having persistent thoughts about hurting myself in serious ways.", "harmful_thoughts"),
("There are moments when I think about ending it all just to stop the pain.", "harmful_thoughts"),


# Anxiety
("My mind won't stop racing with worry, it's exhausting.", "anxiety"),
("Feels like I'm always on edge, waiting for something bad to happen.", "anxiety"),
("This constant anxiety is making everyday life really difficult.", "anxiety"),
("I'm so anxious about the presentation tomorrow, I can hardly sleep.", "anxiety"),
("The thought of going to the party alone is filling me with dread.", "anxiety"),
("Every little sound makes me jump because of my anxiety.", "anxiety"),
("I get anxious over things that probably won't even happen.", "anxiety"),
("Just thinking about my to-do list makes me feel panicked.", "anxiety"),
("I avoid social situations because they make me feel anxious.", "anxiety"),
("I have this unexplainable fear that everyone is judging me.", "anxiety"),
("I can't seem to shake off this feeling of dread.", "anxiety"),
("Lying in bed at night, my thoughts spiral out of control.", "anxiety"),
("Just the thought of going out makes my heart race.", "anxiety"),
("I find myself worrying over things that I know shouldn't matter.", "anxiety"),
("Every small task feels overwhelming, like I'm climbing a mountain.", "anxiety"),
("I keep getting these panic attacks out of nowhere.", "anxiety"),
("It feels like my mind is a prison I can't escape from.", "anxiety"),
("I'm always anticipating the worst, even when things are going well.", "anxiety"),
("My heart pounds in my chest for no apparent reason.", "anxiety"),
("I avoid social situations because I'm afraid I'll say something wrong.", "anxiety"),
("I'm constantly checking things because I'm scared something bad will happen.", "anxiety"),
("I feel like I'm on the verge of a breakdown, but I don't know why.", "anxiety"),
("It's hard to concentrate when your mind is filled with what-ifs.", "anxiety"),
("I'm scared of losing control over myself or my life.", "anxiety"),
("Sometimes, I feel a sudden wave of panic and I don't know why.", "anxiety"),
("I obsess over things I've said or done, worried I made a mistake.", "anxiety"),
("I feel trapped in a cycle of anxiety, and I don't know how to break free.", "anxiety"),
("The world feels like it's closing in on me, suffocating me.", "anxiety"),
("I get tense and jittery at the thought of the future.", "anxiety"),
("My anxiety keeps me up at night, leaving me exhausted during the day.", "anxiety"),


# Depression
("I just can't shake off this heavy feeling in my heart.", "depression"),
("Waking up in the morning feels increasingly difficult.", "depression"),
("I feel like I'm walking through life in a fog.", "depression"),
("Lately, it feels like the color has drained from the world.", "depression"),
("I've lost interest in hobbies and activities I used to love.", "depression"),
("It's like there's a weight on my chest that won't go away.", "depression"),
("I feel isolated, even when I'm with people.", "depression"),
("Smiling feels like a monumental task these days.", "depression"),
("I keep wondering if I'll ever feel like myself again.", "depression"),
("There are days when getting out of bed feels impossible.", "depression"),



]

# Preprocess Test Sentences and Convert to BoW
test_x = []
test_y = []
for sentence, label in test_sentences:
    bow = bag_of_words(sentence)
    test_x.append(bow)
    # Assuming your classes array is a list of intent tags
    test_y.append(classes.index(label))

test_x = np.array(test_x)
# Convert test_y to categorical for evaluation
from tensorflow.keras.utils import to_categorical
test_y_categorical = to_categorical(test_y, num_classes=len(classes))

# Evaluate Model on Test Data
test_loss, test_accuracy = model.evaluate(test_x, test_y_categorical, verbose=1)
print(f'Test Accuracy: {test_accuracy*100:.2f}%')

# Optional: Predictions and Confusion Matrix
predictions = model.predict(test_x)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_y

# Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))

# Adjusting Classification Report to Match Test Data Classes
unique_classes = np.unique(np.concatenate((true_classes, predicted_classes)))

target_names_filtered = [classes[i] for i in unique_classes]

# Classification Report with Filtered Target Names
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, labels=unique_classes, target_names=target_names_filtered))

14/14 [==============================] - 0s 1ms/step - loss: 2.4605 - accuracy: 0.6196
Test Accuracy: 61.96%
14/14 [==============================] - 0s 665us/step
Confusion Matrix:
[[3 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 3 ... 0 0 0]
 ...
 [0 0 0 ... 5 0 0]
 [0 0 0 ... 0 5 0]
 [0 0 0 ... 0 0 6]]

Classification Report:
                  precision    recall  f1-score   support

           about       0.43      0.60      0.50         5
       afternoon       1.00      0.40      0.57         5
          amused       1.00      1.00      1.00         3
         anxiety       0.68      0.50      0.58        30
         ashamed       0.70      0.88      0.78         8
           bored       1.00      0.67      0.80         3
            calm       0.30      0.33      0.32         9
       confident       0.50      0.67      0.57         3
        confused       0.62      1.00      0.76         8
         content       0.62      0.90      0.73        20
         curious       0.60      1.